In [1]:
#!pip install sklearn-recommender
#!pip install sklearn-evaluation
#!pip install black
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
import time
import joblib
import sklearn_recommender as recommenders
import sklearn_evaluation as evaluation
print("hlo")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\deekshitha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\deekshitha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\deekshitha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\deekshitha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\deekshitha\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\deekshitha\AppData\Roaming\nltk_data...
[nltk_data] 

hlo


In [2]:
song_df_1=pd.read_table("million_song.txt",header=None)
song_df_1.columns=['user_id','song_id','listen_count']
song_df_2=pd.read_csv("song_data.csv")
#song_df_1.head()
#song_df_2.head()
song_df=pd.merge(song_df_1, song_df_2.drop_duplicates(['song_id']), on="song_id", how="left")
#song_df.head()
print(len(song_df_1))
print(len(song_df_2))
len(song_df)

2000000
1000000


2000000

In [3]:
song_df['song'] = song_df['title'].map(str) + " - " + song_df['artist_name']

In [4]:
unique_artists=song_df['artist_name'].unique()
print(len(unique_artists))

3375


In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
song_df = song_df.sample(n=1000).drop('title', axis=1).reset_index(drop=True)
#song_df['artist_name'] = song_df['artist_name'].str.replace(r'\n', '')
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')
tfidf_matrix = tfidf.fit_transform(song_df['artist_name'])

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarities = cosine_similarity(tfidf_matrix) 

similarities = {}
for i in range(len(cosine_similarities)):
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    similarities[song_df['song'].iloc[i]] = [(cosine_similarities[i][x], song_df['song'][x], 
                                           song_df['artist_name'][x]) for x in similar_indices][1:]

In [8]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix
    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)       
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------") 
    def recommend(self, recommendation):
            # Get song to find recommendations for
            song = recommendation['song']
            # Get number of songs to recommend
            number_songs = recommendation['number_songs']
            # Get the number of songs most similars from matrix similarities
            recom_song = self.matrix_similar[song][:number_songs]
            # print each item
            self._print_message(song=song, recom_song=recom_song)

In [9]:
recommedations = ContentBasedRecommender(similarities)

In [10]:
recommendation = {
    "song": song_df['song'].iloc[100],
    "number_songs": 7
}

In [11]:
recommedations.recommend(recommendation)

The 7 recommended songs for Walking On The Moon - The Police are:
Number 1:
Driven To Tears - The Police by The Police with 1.0 similarity score
--------------------
Number 2:
O My God - The Police by The Police with 1.0 similarity score
--------------------
Number 3:
Walking On The Moon - The Police by The Police with 1.0 similarity score
--------------------
Number 4:
Walking On The Moon - The Police by The Police with 1.0 similarity score
--------------------
Number 5:
Bulletproof - La Roux by La Roux with 0.0 similarity score
--------------------
Number 6:
Jimbo (Album Version) - Eric Burdon & War by Eric Burdon & War with 0.0 similarity score
--------------------
Number 7:
Real World  (LP Version) - matchbox twenty by matchbox twenty with 0.0 similarity score
--------------------
